Can we train a machine learning model on top of the neural network that improves the Q2 prediction from a model traind with multiple time steps?

In [ ]:
import torch
from uwnet.columns import single_column_simulation

ds = xr.open_dataset("../data/processed/training.nc").isel(y=slice(24, 40))
model = torch.load("../models/150/2.pkl")

output = model.call_with_xr(ds)

dt = .125 * 86400

q1 = (ds.SLI.shift(time=-1) - ds.SLI)/dt - (ds.FSLI + ds.FSLI.shift(time=-1))/2
q2 = (ds.QT.shift(time=-1) - ds.QT)/dt - (ds.FQT + ds.FQT.shift(time=-1))/2
q2 = q2.dropna('time')

# Compare Q1 and predicted Q1

In [ ]:
output.SLI[:,:,8,0].plot(x='time')

In [ ]:
q1[:,:,8,0].plot(x='time')

There are much less problems with the temperature field.

# Compare Q2 and predicted Q2

In [ ]:
output.QT.isel(y=8, x=0).plot(x='time')

Here is the true value of Q2.

In [ ]:
q2.isel(y=8, x=0).plot(x='time')

Again there are problems with the humidity in the upper troposphere.

# Debiasing using Lasso

In [ ]:
x, y = xr.broadcast(output.QT[:-1]/86400, q2)

x = x.stack(samples=['y', 'x', 'time']).values.T
y = y.stack(samples=['y', 'x', 'time']).values.T

n,m =x.shape

subset = 100000

inds = np.random.choice(n, subset)

x = x.take(inds, axis=0)
y = y.take(inds, axis=0)

from sklearn.linear_model import LinearRegression, Lasso, Ridge

mod = Lasso(fit_intercept=True, alpha=1.2e-10).fit(x, y)
print(mod.score(x, y))
a = mod.coef_

plt.pcolormesh(a)
plt.colorbar()

np.abs(np.linalg.eigvals(a)).max()

This shows pretty good results. As long as the maximum eigenvalue of this matrix is less than 0, I assume that it will produce good predictions, and should be stable after running for multiple time steps.

In [ ]:
qt_loc = output.QT[:,:,8,0].values
qt_pred_lasso = mod.predict(qt_loc[:-1])

In [ ]:
def plot(x):
    plt.pcolormesh(output.time, output.z, x.T, vmin=-20, vmax=20, cmap='RdBu_r')

plot(qt_pred_lasso)
plt.colorbar()

## Scatter origina model + LASSO vs Q2

In [ ]:
q2_loc = q2.isel(y=8, x=0).values

In [ ]:
plt.scatter(qt_pred_lasso, q2_loc.ravel()*86400, alpha=.01)
plt.plot([-30, 30], [-30, 30], c='k')

## Scatter plot of the original model vs Q2

In [ ]:
plt.scatter(qt_loc[:-1].ravel(), q2_loc.ravel()*86400, alpha=.01)
plt.plot([-30, 30], [-30, 30], c='k')

The LASSO signifnicantly hurts the instanstaeneous prediction of the scheme.